#  Collections Data to Linked Art - Cleveland Museum of Art

## Import What We Need for Notebook
See the [Transform Introduction](01-00-Transform-Intro.ipynb) for a description of the Python libraries used

In [ ]:
try:
    import IPython
except:
    !pip install IPython
    import IPython   
    
from IPython.display import display,IFrame,HTML
 
try:
    import json
except:
    !pip install json
    import json 
  
try:
    import csv
except:
    !pip install csv
    import csv
    
import os

import cromulent
from cromulent.model import factory
    
from lib import linkedart as la       

## Input Data
The input data file is from https://github.com/ClevelandMuseumArt/openaccess


### Parse CSV File

In [6]:


file = './data/cma/input/data.csv'

#remove BOM
s = open(file, mode='r', encoding='utf-8-sig').read()
open(file, mode='w', encoding='utf-8').write(s)

allObjects = csv.DictReader(open(file, mode='r',encoding='utf-8'))

for obj in allObjects:
    print(json.dumps(obj,indent=2))
    break  

{
  "id": "74539",
  "accession_number": "2015.449",
  "share_license_status": "CC0",
  "tombstone": "A Miller's Carriage, c. 1895. Albert-Charles Lebourg (French, 1849-1928). Black and white chalk with stumping ; sheet: 33.2 x 49.7 cm (13 1/16 x 19 9/16 in.). The Cleveland Museum of Art, Bequest of Muriel Butkin 2015.449",
  "current_location": "",
  "title": "A Miller's Carriage",
  "title_in_original_language": "",
  "series": "",
  "series_in_original_language": "",
  "creation_date": "c. 1895",
  "creation_date_earliest": "1890",
  "creation_date_latest": "1900",
  "creators": "Albert-Charles Lebourg (French, 1849-1928), artist",
  "culture": "France, 19th-20th century",
  "technique": "Black and white chalk with stumping ",
  "support_materials": "gray laid paper",
  "department": "Drawings",
  "collection": "DR - French",
  "type": "Drawing",
  "measurements": "Sheet: 33.2 x 49.7 cm (13 1/16 x 19 9/16 in.)",
  "state_of_the_work": "",
  "edition_of_the_work": "",
  "creditline":

### Create field mapping

In [22]:
mapp =  {
    "id":"id",
    "accession_number":"accession_number",
    "accession_date": "",
    "classification" : "type",
    "title": "title",
    "alt_title": "title_in_original_language",
    "notes": "tombstone",
    "date_created":"creation_date",
    "date_created_earliest": "creation_date_earliest",
    "date_created_latest": "creation_date_latest",
    "created_period":"culture",
    "created_dynasty":"",
    "created_inscriptions":"inscriptions",
    "created_notes": "fun_fact",
    "creator":"creators",
    "physical_medium": "Medium",
    "physical_style": "",
    "physical_technique": "technique",
    "physical_description": "",
    "physical_dimensions": "measurements",
    "created_provenance": "provenance" ,
    "credit_line": "creditline",
    "collection" : "department",
    "current_status" : "current_location",
    "current_location": "current_location"
    
}




### Create object property dictionary

In [28]:
def createObjProp(obj,mapp):
    objProp = {}
    csv_keys = list(obj.keys())
    for key in csv_keys:
        for prop in mapp:
            if key == mapp[prop]:
                if prop == "creator":
                    objProp[prop] = {"name": obj[key]}
                else:
                    objProp[prop] = obj[key]
    
    objProp["current_owner"] = {"name":"Cleveland Museum of Art",
                                "location":"Cleveland,Ohio",
                                "type": "http://vocab.getty.edu/aat/300312281" ,
                                "type_label": ""}
    return objProp


#  baseURI for JSON-LD document
baseURI = "https://clevelandart.org/art/"


for obj in allObjects:
    objProp = createObjProp(obj,mapp)
    id = obj[mapp.get("id")]
    object_uri = baseURI + id
    
    objLA = None
    objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
    # write to file 
    text_file = open("./data/cma/output/json/all/" + id + ".json", "wt")
    n = text_file.write(factory.toString(objLA, compact=False))
    text_file.close()

In [31]:
def fn(fpath):       # 1.Get file names from directory
    file_list=os.listdir(r"" + fpath)
    cnt =1 
    for file in file_list:
        cnt = cnt+1
        if cnt > 10:
            break
        display(HTML("<a target='_new' href='" + fpath + file + "'>" + file + "</a>"))

fn("./data/cma/output/json/all/")